Import Statements for Pandas, Numpy, SKLearn

In [350]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

Read in the datasets

In [351]:
train = pd.read_csv("train.csv", sep=",")
test = pd.read_csv("test.csv",sep=",")

In [352]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [353]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [354]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Drop unnecessary columns that won't impact /  or include null values

In [355]:
train.drop(['Name', 'Cabin', 'Ticket', 'Age'], axis = 'columns', inplace = True )
test.drop(['Name', 'Cabin', 'Ticket', 'Age'], axis = 'columns', inplace = True )

Encode the datasets with OneHot Encoder

In [356]:
traindummies = pd.get_dummies(train)
testdummies = pd.get_dummies(test)

In [357]:
traindummies.head()

,PassengerId,Survived,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,1,0,7.2500,0,1,0,0,1
1,2,1,1,1,0,71.2833,1,0,1,0,0
2,3,1,3,0,0,7.9250,1,0,0,0,1
3,4,1,1,1,0,53.1000,1,0,0,0,1
4,5,0,3,0,0,8.0500,0,1,0,0,1


In [358]:
testdummies.head()

,PassengerId,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,0,0,7.8292,0,1,0,1,0
1,893,3,1,0,7.0000,1,0,0,0,1
2,894,2,0,0,9.6875,0,1,0,1,0
3,895,3,0,0,8.6625,0,1,0,0,1
4,896,3,1,1,12.2875,1,0,0,0,1


See the correlation of Survived at the base

In [359]:
traindummies.corr()['Survived']

PassengerId   -0.005007
Survived       1.000000
Pclass        -0.338481
SibSp         -0.035322
Parch          0.081629
Fare           0.257307
Sex_female     0.543351
Sex_male      -0.543351
Embarked_C     0.168240
Embarked_Q     0.003650
Embarked_S    -0.155660
Name: Survived, dtype: float64

In [360]:
traindummies.shape

(891, 11)

In [361]:
traindummies = traindummies.dropna()

In [362]:
traindummies.shape

(891, 11)

Test Dummies can't drop the null in fare or it won't have enough rows to turn in on kaggle

In [377]:
#testdummies = testdummies.dropna()
testdummies.shape

(418, 10)

Check the columns to locate Null values

In [378]:
testdummies.isna().sum()

PassengerId    0
Pclass         0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [365]:
#testdummies = testdummies.dropna()
#If I drop NA I won't be able to submit this because missing one row.
testdummies['Fare']= np.nan_to_num(testdummies['Fare'])

In [379]:
# Check to see if the null in fare is fixed
testdummies.isna().sum()

PassengerId    0
Pclass         0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

Make an X and Y train

In [380]:
# Use the column survived in the training set
X_train, Y_train = traindummies.drop(columns=['Survived']), traindummies['Survived']


In [368]:
X_train.head()

,PassengerId,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,3,1,0,7.2500,0,1,0,0,1
1,2,1,1,0,71.2833,1,0,1,0,0
2,3,3,0,0,7.9250,1,0,0,0,1
3,4,1,1,0,53.1000,1,0,0,0,1
4,5,3,0,0,8.0500,0,1,0,0,1


In [369]:
Y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

Starting off with random forest
Create forest and add the X, Y

In [370]:
forest = RandomForestClassifier(n_estimators=100)
forest.fit(X_train, Y_train)

RandomForestClassifier()

Create Method to predict survival

Update: Have to fix the method so that all the NaN values in survived get put as numbers so it will run since we are not dropping any rows from test dummies.
Update: Nan_to_Num here did not work so switched to top where test was having a single NAN

In [371]:
def predict_survival(tree, test):
    submission = pd.DataFrame()
    #Copy Passenger Column from Test DF
    submission['PassengerId'] = test['PassengerId']
    #Make a Survived Prediction column 
    #Edit Needs to be labeled Survived or you won't be able to submit predictions
    #submission['Survived_Prediction'] = tree.predict(test).round().astype(int)
    submission['Survived'] = tree.predict(test).round().squeeze().astype(int)
    print(submission)
    submission.to_csv("submission.csv", index=False)

In [372]:
#Not enough rows for submission
predict_survival(forest, testdummies)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]


77% Submission

Decision Tree

In [373]:
dtree = DecisionTreeClassifier(random_state=0)
dtree.fit(X_train, Y_train)

DecisionTreeClassifier(random_state=0)

In [374]:
predict_survival(dtree, testdummies)

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


71% Accuracy

In [375]:
dtreerandom = DecisionTreeClassifier(random_state=100)
dtreerandom.fit(X_train, Y_train)

DecisionTreeClassifier(random_state=100)

In [376]:
predict_survival(dtreerandom, testdummies)

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


70%